In [1]:
# %reload_ext autoreload
# %autoreload 2

# import os
# import sys

# # Add the root folder to the module search path
# # Get the current directory
# current_directory = os.getcwd()

# # Move two levels up (go to the parent directory of the parent directory)
# two_levels_up_directory = os.path.dirname(os.path.dirname(current_directory))

# print(two_levels_up_directory)

# sys.path.append(two_levels_up_directory)

In [2]:
import pandas as pd
from pykoi.rlhf import RLHFConfig
from pykoi.rlhf import RewardFinetuning
import mlflow
import datetime

/home/ubuntu/miniconda3/envs/koi/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Train RLHF using the data from database

Let's take a look of the QA data and process it for training.

In [ ]:
input_root = "input/"
input_file = "qd_immigration.csv"
stack_exchange_df = pd.read_csv(input_root + input_file)
stack_exchange_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'input/qd_immigration.csv'

In [ ]:
from datasets import Dataset

print("My ranking database has {} samples".format(stack_exchange_df.shape[0]))
dataset = Dataset.from_dict(stack_exchange_df)
dataset

My ranking database has 45 samples


Dataset({
    features: ['Question', 'Answer'],
    num_rows: 45
})

### Train with RLHF

In [ ]:
# Set up mlflow experiment name.

# mlflow.set_tracking_uri("http://x.x.x.x:5000")
experiment = "rlhf_step2_reward"
current_time = str(datetime.datetime.now())
mlflow_experiment_name = '/'.join([experiment, current_time])

try:
    mlflow.end_run()
except:
    print("No mlflow run in progress")

mlflow.set_experiment(mlflow_experiment_name)

2023/10/06 17:43:16 INFO mlflow.tracking.fluent: Experiment with name 'rlhf_step2_reward/2023-10-06 17:43:16.276345' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///home/ubuntu/pykoi/example/rlhf/mlflow/mlruns/928497460767897508', creation_time=1696614196292, experiment_id='928497460767897508', last_update_time=1696614196292, lifecycle_stage='active', name='rlhf_step2_reward/2023-10-06 17:43:16.276345', tags={}>

In [ ]:
# Set pykoi parameters.
reward_model_path = "elinas/llama-7b-hf-transformers-4.29"
dataset_type = "local_db"
trained_model_path = "./models/rlhf_step2_rw"

In [ ]:
# Manually log pykoi parameters into mlflow. Torch level parameters are automatically logged.
mlflow.log_param("pykoi_reward_model_path", reward_model_path)
mlflow.log_param("pykoi_dataset_type", dataset_type)
mlflow.log_param("pykoi_trained_model_path", trained_model_path)

'./models/rlhf_step2_rw'

In [ ]:
# Run supervised finetuning
# Training metrics are automatically logged into mlflow.
config = RLHFConfig(reward_model_path=reward_model_path, 
                          dataset_type=dataset_type,
                          )
rlhf_step2_rft = RewardFinetuning(config)
rlhf_step2_rft.train_and_save(trained_model_path)


Loading checkpoint shards: 100%|██████████| 2/2 [01:48<00:00, 54.48s/it]
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at elinas/llama-7b-hf-transformers-4.29 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 8,396,800 || all params: 6,615,740,416 || trainable%: 0.12692154576821896


ValueError: Length mismatch: Expected axis has 0 elements, new values have 4 elements

In [ ]:
# Save the trained reward model and input into mlflow artifacts.
mlflow.log_artifacts(trained_model_path)
mlflow.log_artifacts(input_root)

In the terminal, run
```
mlflow ui
```
and go to http://127.0.0.1:5000 in the browser to view the experiment in the UI.